In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

from catboost import CatBoostClassifier

In [6]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Чтение данных

In [7]:
reports = pd.read_csv('report_dates.csv')
clients = pd.read_csv('clients.csv')
train = pd.read_csv('train.csv')
transactions = pd.read_csv('transactions.csv')

In [8]:
reports = reports.set_index('report')
reports['report_dt'] = reports.report_dt.astype('datetime64[ns]')

In [9]:
reports

,report_dt
report,
1,2022-07-31 03:00:00
2,2022-08-31 03:00:00
3,2022-09-30 03:00:00
4,2022-10-31 03:00:00
5,2022-11-30 03:00:00
6,2022-12-31 03:00:00
7,2023-01-31 03:00:00
8,2023-02-28 03:00:00
9,2023-03-31 03:00:00


In [10]:
clients = clients.set_index('user_id')
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':0,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)
clients = clients.join(reports, on='report', how='left')

In [11]:
clients

,report,employee_count_nm,bankemplstatus,customer_age,report_dt
user_id,,,,,
3,2,4.0,0,3,2022-08-31 03:00:00
9,1,6.0,0,3,2022-07-31 03:00:00
13,6,5.0,0,2,2022-12-31 03:00:00
37,5,6.0,0,2,2022-11-30 03:00:00
41,1,4.0,0,2,2022-07-31 03:00:00
...,...,...,...,...,...
562043,12,NaN,0,2,2023-06-30 03:00:00
562205,12,NaN,0,1,2023-06-30 03:00:00
562312,12,NaN,0,0,2023-06-30 03:00:00


In [12]:
clients.employee_count_nm.value_counts()

employee_count_nm
6.0    17833
4.0    14362
5.0     8477
2.0     7751
3.0     7314
0.0     3797
Name: count, dtype: int64

In [13]:
clients.employee_count_nm.info()

<class 'pandas.core.series.Series'>
Index: 96000 entries, 3 to 562740
Series name: employee_count_nm
Non-Null Count  Dtype  
--------------  -----  
59534 non-null  float64
dtypes: float64(1)
memory usage: 1.5 MB


In [14]:
train = train.set_index('user_id')

In [15]:
train

,target,time
user_id,,
3,0,77
13,0,86
37,0,89
41,0,57
42,0,84
...,...,...
561824,0,91
562043,0,75
562312,0,91


In [16]:
transactions = transactions.rename(columns={'transaction_amt': 'amt', 'transaction_dttm': 'dttm'})
transactions['dttm'] = transactions.dttm.astype('datetime64[ns]')

In [17]:
transactions

,user_id,mcc_code,currency_rk,amt,dttm
0,3,3,1,-183.883957,2022-01-28 12:05:33
1,3,3,1,-3206.437012,2022-01-28 12:52:30
2,3,16,1,-153866.890625,2022-02-16 14:45:56
3,3,56,1,-15144.601562,2022-03-09 19:58:29
4,3,0,1,5297.908691,2022-03-12 18:11:31
...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49


# Создание дополнительных переменных

In [18]:
def logsumabs(x):
    return np.log(x.abs().sum())

In [19]:
transactions = transactions.join(clients, on='user_id', how='left')

In [20]:
transactions.head(5)

,user_id,mcc_code,currency_rk,amt,dttm,report,employee_count_nm,bankemplstatus,customer_age,report_dt
0,3,3,1,-183.883957,2022-01-28 12:05:33,2,4.0,0,3,2022-08-31 03:00:00
1,3,3,1,-3206.437012,2022-01-28 12:52:30,2,4.0,0,3,2022-08-31 03:00:00
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2,4.0,0,3,2022-08-31 03:00:00
3,3,56,1,-15144.601562,2022-03-09 19:58:29,2,4.0,0,3,2022-08-31 03:00:00
4,3,0,1,5297.908691,2022-03-12 18:11:31,2,4.0,0,3,2022-08-31 03:00:00


In [21]:
transactions['date'] = transactions.dttm.dt.date.astype('datetime64[ns]')
transactions['dweek'] = transactions.dttm.dt.dayofweek

transactions['d'] = (transactions.report_dt - transactions.dttm).dt.days
transactions['w'] = transactions.d // 7
transactions['m'] = transactions.report_dt.dt.month - transactions.dttm.dt.month

transactions['interval'] = transactions.groupby('user_id').date.diff().dropna().dt.days

In [22]:
transactions['p_amt'] = transactions.amt.apply(lambda x: x if x > 0 else 0)
transactions['n_amt'] = transactions.amt.apply(lambda x: x if x < 0 else 0)

In [23]:
transactions.head(5)

,user_id,mcc_code,currency_rk,amt,dttm,report,employee_count_nm,bankemplstatus,customer_age,report_dt,date,dweek,d,w,m,interval,p_amt,n_amt
0,3,3,1,-183.883957,2022-01-28 12:05:33,2,4.0,0,3,2022-08-31 03:00:00,2022-01-28,4,214,30,7,NaN,0.000000,-183.883957
1,3,3,1,-3206.437012,2022-01-28 12:52:30,2,4.0,0,3,2022-08-31 03:00:00,2022-01-28,4,214,30,7,0.0,0.000000,-3206.437012
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2,4.0,0,3,2022-08-31 03:00:00,2022-02-16,2,195,27,6,19.0,0.000000,-153866.890625
3,3,56,1,-15144.601562,2022-03-09 19:58:29,2,4.0,0,3,2022-08-31 03:00:00,2022-03-09,2,174,24,5,21.0,0.000000,-15144.601562
4,3,0,1,5297.908691,2022-03-12 18:11:31,2,4.0,0,3,2022-08-31 03:00:00,2022-03-12,5,171,24,5,3.0,5297.908691,0.000000


In [24]:
%%time
tab1 = transactions.groupby(['user_id','currency_rk']).amt.agg([logsumabs])
tab1 = tab1.unstack('currency_rk').fillna(0)
tab1.columns = ['ELSE','RUR','USD','EUR']

CPU times: user 7.05 s, sys: 144 ms, total: 7.19 s
Wall time: 7.22 s


In [25]:
tab1.head(5)

,ELSE,RUR,USD,EUR
user_id,,,,
3,0.00000,12.789712,0.0,0.0
9,0.00000,12.686752,0.0,0.0
13,9.28478,12.825245,0.0,0.0
37,0.00000,12.775178,0.0,0.0
41,0.00000,11.595303,0.0,0.0


In [26]:
transactions.currency_rk.value_counts()

currency_rk
1    12701022
0      370455
3        1795
2        1751
Name: count, dtype: int64

In [27]:
%%time
tab2 = (transactions
        .groupby('user_id')
        .agg(
            tran_n = ('amt', lambda x: x.count()),
            amt_sum = ('amt', lambda x: x.sum()),
            amt_max = ('amt', lambda x: x.max()),
            amt_min = ('amt', lambda x: x.min()),
            amt_mean = ('amt', lambda x: x.mean()),
            amt_q_10 = ('amt', lambda x: x.quantile(0.1)),
            amt_q_90 = ('amt', lambda x: x.quantile(0.9)),
            amt_median = ('amt', lambda x: x.median()),
            amt_std = ('amt', lambda x: x.std()),
            amt_kurtosis = ('amt', lambda x: x.kurtosis()),
            amt_skew = ('amt', lambda x: x.skew())
        ))

CPU times: user 1min 19s, sys: 662 ms, total: 1min 19s
Wall time: 1min 20s


In [28]:
tab2.head(5)

,tran_n,amt_sum,amt_max,amt_min,amt_mean,amt_q_10,amt_q_90,amt_median,amt_std,amt_kurtosis,amt_skew
user_id,,,,,,,,,,,
3,11,13706.416641,104011.960938,-153866.890625,1246.037876,-15144.601562,37991.929688,4549.455078,60860.880477,5.091569,-1.399640
9,90,-323434.666813,-45.579891,-90147.617188,-3593.718520,-4270.121582,-219.823502,-840.509674,11797.055119,37.221420,-5.877426
13,22,-124717.379150,70322.828125,-58740.300781,-5668.971780,-23855.722852,10937.443262,-10529.004883,23507.953991,5.245836,1.172701
37,315,-331859.599463,5487.140625,-35782.984375,-1053.522538,-1940.371729,-74.076974,-236.420776,3913.114835,41.551736,-6.201203
41,16,-108586.614166,-290.766998,-16841.208984,-6786.663385,-12649.590820,-531.802338,-6328.293701,5251.737982,-1.001280,-0.291520


In [29]:
%%time
tab3 = (transactions
        .groupby('user_id')
        .agg(
            mcc_nunique = ('mcc_code', lambda x: x.nunique())
        ))

CPU times: user 3.57 s, sys: 74.4 ms, total: 3.65 s
Wall time: 3.66 s


In [30]:
tab3.head(5)

,mcc_nunique
user_id,
3,4
9,22
13,4
37,28
41,5


In [31]:
%%time
tab4 = (transactions
        .groupby('user_id')
        .agg(
            last_day_diff = ('d', lambda x: min(x)),
            first_day_diff = ('d', lambda x: max(x)),
            life = ('d', lambda x: max(x) - min(x)),
            interval_max = ('interval', lambda x: x.max()),
            interval_min = ('interval', lambda x: x.min()),
            interval_mean = ('interval', lambda x: x.mean()),
            interval_median = ('interval', lambda x: x.median()),
            interval_q_10 = ('interval', lambda x: x.quantile(0.1)),
            interval_q_90 = ('interval', lambda x: x.quantile(0.9)),
            interval_kurtosis = ('interval', lambda x: x.kurtosis()),
            interval_skew = ('interval', lambda x: x.skew()),
            interval_std = ('interval', lambda x: x.std())
        ))

CPU times: user 1min 19s, sys: 630 ms, total: 1min 20s
Wall time: 1min 20s


In [32]:
tab4.head(5)

,last_day_diff,first_day_diff,life,interval_max,interval_min,interval_mean,interval_median,interval_q_10,interval_q_90,interval_kurtosis,interval_skew,interval_std
user_id,,,,,,,,,,,,
3,108,214,106,30.0,0.0,10.600000,7.5,0.0,21.9,-0.946217,0.603867,10.689559
9,102,283,181,22.0,0.0,2.033708,1.0,0.0,6.0,12.504731,3.340798,3.791286
13,114,282,168,21.0,0.0,8.000000,8.0,0.0,18.0,-1.223967,0.457177,7.280110
37,104,283,179,7.0,0.0,0.570064,0.0,0.0,1.0,14.055624,3.109092,0.947434
41,103,256,153,31.0,0.0,10.200000,2.0,0.0,29.2,-1.427902,0.815989,13.039500


In [33]:
%%time
tab5 = (transactions
        .eval('log_p_amt = log(p_amt + 1)')
        .eval('log_n_amt = log(-n_amt + 1)')
        .groupby('user_id')
        .agg(
            sp = ('log_p_amt', lambda x: sum(x)),
            sn = ('log_n_amt', lambda x: sum(x)),
            p_count = ('p_amt', lambda x: x[x>0].count()),
            n_count = ('n_amt', lambda x: x[x<0].count()),
            p_mean = ('log_p_amt', lambda x: x.mean()),
            n_mean = ('log_n_amt', lambda x: x.mean())
        )
        .eval('balance = sp - sn')
       )

CPU times: user 35 s, sys: 8.39 s, total: 43.4 s
Wall time: 42.7 s


In [34]:
tab5.head(5)

,sp,sn,p_count,n_count,p_mean,n_mean,balance
user_id,,,,,,,
3,64.222449,34.862271,7,4,5.838404,3.169297,29.360178
9,0.000000,622.581098,0,90,0.000000,6.917568,-622.581098
13,73.236937,135.005210,8,14,3.328952,6.136600,-61.768272
37,17.176832,1772.637183,2,313,0.054530,5.627420,-1755.460350
41,0.000000,132.182240,0,16,0.000000,8.261390,-132.182240
...,...,...,...,...,...,...,...
562043,0.000000,208.525761,0,37,0.000000,5.635831,-208.525761
562205,17.145452,709.602714,3,148,0.113546,4.699356,-692.457262
562312,0.000000,302.465883,0,56,0.000000,5.401176,-302.465883


In [35]:
%%time
tab6 = transactions.pivot_table(    
    values = 'amt',
    index = 'user_id',
    columns = 'mcc_code',
    aggfunc = [logsumabs, 'count'],
    fill_value=0)
tab6.columns = list(map(lambda x: 'mcc_' + str(x[0]) + '_[' + str(x[1]) + ']', tab6.columns))

CPU times: user 1min 35s, sys: 1.99 s, total: 1min 37s
Wall time: 1min 36s


In [36]:
tab6.head(5)

,mcc_logsumabs_[0],mcc_logsumabs_[1],mcc_logsumabs_[2],mcc_logsumabs_[3],mcc_logsumabs_[4],mcc_logsumabs_[6],mcc_logsumabs_[7],mcc_logsumabs_[8],mcc_logsumabs_[9],mcc_logsumabs_[10],mcc_logsumabs_[11],mcc_logsumabs_[12],mcc_logsumabs_[13],mcc_logsumabs_[14],mcc_logsumabs_[15],mcc_logsumabs_[16],mcc_logsumabs_[17],mcc_logsumabs_[18],mcc_logsumabs_[19],mcc_logsumabs_[20],mcc_logsumabs_[21],mcc_logsumabs_[22],mcc_logsumabs_[23],mcc_logsumabs_[24],mcc_logsumabs_[25],mcc_logsumabs_[26],mcc_logsumabs_[27],mcc_logsumabs_[28],mcc_logsumabs_[29],mcc_logsumabs_[30],mcc_logsumabs_[31],mcc_logsumabs_[32],mcc_logsumabs_[33],mcc_logsumabs_[34],mcc_logsumabs_[35],mcc_logsumabs_[36],mcc_logsumabs_[37],mcc_logsumabs_[38],mcc_logsumabs_[39],mcc_logsumabs_[40],mcc_logsumabs_[41],mcc_logsumabs_[42],mcc_logsumabs_[43],mcc_logsumabs_[44],mcc_logsumabs_[45],mcc_logsumabs_[46],mcc_logsumabs_[47],mcc_logsumabs_[48],mcc_logsumabs_[49],mcc_logsumabs_[50],...,mcc_count_[292],mcc_count_[293],mcc_count_[294],mcc_count_[296],mcc_count_[297],mcc_count_[298],mcc_count_[300],mcc_count_[302],mcc_count_[303],mcc_count_[304],mcc_count_[305],mcc_count_[306],mcc_count_[307],mcc_count_[309],mcc_count_[310],mcc_count_[311],mcc_count_[312],mcc_count_[313],mcc_count_[314],mcc_count_[316],mcc_count_[317],mcc_count_[318],mcc_count_[322],mcc_count_[323],mcc_count_[325],mcc_count_[326],mcc_count_[327],mcc_count_[328],mcc_count_[330],mcc_count_[332],mcc_count_[334],mcc_count_[338],mcc_count_[340],mcc_count_[342],mcc_count_[346],mcc_count_[356],mcc_count_[360],mcc_count_[364],mcc_count_[368],mcc_count_[371],mcc_count_[372],mcc_count_[382],mcc_count_[389],mcc_count_[392],mcc_count_[407],mcc_count_[412],mcc_count_[413],mcc_count_[424],mcc_count_[434],mcc_count_[449]
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,12.134084,0.000000,0.000000,8.128680,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,11.943843,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0.000000,9.930944,9.026902,12.168032,7.417504,7.619438,0.0000,8.557285,0.000000,0.000000,0.0,0.000000,7.565031,0.0,9.137996,0.000000,0.0,7.577560,0.0,0.0,0.0,0.00000,8.555794,8.978406,6.793393,0.0,0.000000,0.000000,0.0,0.000000,9.292408,6.188763,0.000000,0.0,0.0,0.0,8.225467,0.0,9.706435,0.0,0.0,8.749914,0.0,0.0,0.0,6.295319,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0.000000,0.000000,0.000000,12.437070,0.000000,0.000000,8.6991,0.000000,9.287803,0.000000,0.0,11.639352,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,9.281618,10.489704,10.188235,12.277408,4.843160,6.482128,0.0000,6.938342,10.185608,6.828845,0.0,0.000000,0.000000,0.0,8.000576,6.307584,0.0,5.154446,0.0,0.0,0.0,7.45275,0.000000,0.000000,7.228323,0.0,6.906717,9.608577,0.0,5.034757,0.000000,5.191847,0.000000,0.0,0.0,0.0,7.521133,0.0,8.387374,0.0,0.0,3.597105,0.0,0.0,0.0,7.640647,0.0,0.0,5.674251,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41,0.000000,6.606730,0.000000,11.159380,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,7.506119,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,7.174624,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0

In [37]:
%%time
tab7 = transactions.pivot_table(
    values = 'amt',
    index = 'user_id',
    columns = 'w',
    aggfunc = [logsumabs,'count'],
    fill_value = 0)
tab7.columns = list(map(lambda x: 'w_' + str(x[0]) + '_[' + str(x[1]) + ']', tab7.columns))

CPU times: user 1min 51s, sys: 601 ms, total: 1min 51s
Wall time: 1min 51s


In [38]:
tab7.head(5)

,w_logsumabs_[14],w_logsumabs_[15],w_logsumabs_[16],w_logsumabs_[17],w_logsumabs_[18],w_logsumabs_[19],w_logsumabs_[20],w_logsumabs_[21],w_logsumabs_[22],w_logsumabs_[23],w_logsumabs_[24],w_logsumabs_[25],w_logsumabs_[26],w_logsumabs_[27],w_logsumabs_[28],w_logsumabs_[29],w_logsumabs_[30],w_logsumabs_[31],w_logsumabs_[32],w_logsumabs_[33],w_logsumabs_[34],w_logsumabs_[35],w_logsumabs_[36],w_logsumabs_[37],w_logsumabs_[38],w_logsumabs_[39],w_logsumabs_[40],w_count_[14],w_count_[15],w_count_[16],w_count_[17],w_count_[18],w_count_[19],w_count_[20],w_count_[21],w_count_[22],w_count_[23],w_count_[24],w_count_[25],w_count_[26],w_count_[27],w_count_[28],w_count_[29],w_count_[30],w_count_[31],w_count_[32],w_count_[33],w_count_[34],w_count_[35],w_count_[36],w_count_[37],w_count_[38],w_count_[39],w_count_[40]
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,0.000000,11.602757,10.251875,10.545129,0.000000,0.000000,8.533818,0.000000,0.00000,0.000000,9.925372,0.000000,0.000000,11.943843,0.000000,0.000000,8.128680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,1,1,0,0,2,0,0,0,2,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0
9,8.894026,0.000000,0.000000,8.699712,8.491021,0.000000,8.650308,11.952738,10.76223,0.000000,0.000000,7.934068,8.689353,10.408941,6.865856,9.417162,7.813909,7.648565,8.869627,8.686656,7.950515,8.617455,7.608994,5.005457,7.919247,8.969644,8.079263,2,0,0,6,3,0,6,11,2,0,0,4,3,11,2,5,1,3,6,1,3,2,1,1,3,11,3
13,0.000000,0.000000,10.210958,0.000000,0.000000,9.272584,10.218486,0.000000,0.00000,9.754696,9.251079,0.000000,9.345651,0.000000,0.000000,11.924611,10.112548,0.000000,9.133243,0.000000,9.214152,10.101120,0.000000,9.285050,9.791329,9.284780,9.863950,0,0,2,0,0,1,2,0,0,1,1,0,1,0,0,5,1,0,1,0,2,1,0,1,1,1,1
37,7.616012,5.389874,8.111924,10.425047,9.676393,8.083787,9.280501,9.162937,9.96731,10.573324,8.131733,8.327797,8.240544,10.360541,7.620029,9.659059,9.299723,9.983910,8.652578,8.705591,10.353804,8.023243,10.238417,9.034963,9.227318,10.084929,8.367638,1,1,10,21,15,4,23,22,16,7,18,21,15,10,10,10,23,19,8,11,4,10,9,12,5,3,7
41,8.736538,7.506119,0.000000,0.000000,0.000000,9.781772,0.000000,0.000000,0.00000,9.752138,9.485218,0.000000,0.000000,9.767794,0.000000,0.000000,0.000000,8.798201,9.354683,0.000000,0.000000,0.000000,9.731584,0.000000,0.000000,0.000000,0.000000,1,1,0,0,0,4,0,0,0,2,1,0,0,4,0,0,0,1,1,0,0,0,1,0,0,0,0


In [39]:
%%time
tab8 = transactions.pivot_table(
    values = 'amt',
    index = 'user_id',
    columns = 'm',
    aggfunc = [logsumabs,'count'],
    fill_value = 0)
tab8.columns = list(map(lambda x: 'm_' + str(x[0]) + '_[' + str(x[1]) + ']', tab8.columns))

CPU times: user 38.2 s, sys: 360 ms, total: 38.5 s
Wall time: 38.7 s


In [40]:
tab8.head(5)

,m_logsumabs_[-9],m_logsumabs_[-8],m_logsumabs_[-7],m_logsumabs_[-6],m_logsumabs_[-5],m_logsumabs_[-4],m_logsumabs_[-3],m_logsumabs_[3],m_logsumabs_[4],m_logsumabs_[5],m_logsumabs_[6],m_logsumabs_[7],m_logsumabs_[8],m_logsumabs_[9],m_count_[-9],m_count_[-8],m_count_[-7],m_count_[-6],m_count_[-5],m_count_[-4],m_count_[-3],m_count_[3],m_count_[4],m_count_[5],m_count_[6],m_count_[7],m_count_[8],m_count_[9]
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,11.833084,10.670715,9.925372,11.943843,8.128680,0.000000,0.000000,0,0,0,0,0,0,0,3,3,2,1,2,0,0
9,0.0,0.0,0.0,0.0,9.825543,9.424449,9.313587,9.494728,11.554290,11.606416,10.899446,0.000000,0.000000,0.000000,0,0,0,0,12,9,14,8,15,9,23,0,0,0
13,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10.210958,10.546587,10.227409,11.997725,10.690730,11.063459,9.863950,0,0,0,0,0,0,0,2,3,2,6,4,4,1
37,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10.395736,10.630183,11.192290,10.978041,11.152946,10.820914,10.250063,0,0,0,0,0,0,0,15,77,67,54,55,37,10
41,0.0,0.0,0.0,0.0,9.807809,9.731584,0.000000,8.992861,9.781772,10.320704,9.767794,0.000000,0.000000,0.000000,0,0,0,0,2,1,0,2,4,3,4,0,0,0


# Сборка выборки

In [41]:
df = (clients
      .join(tab1, on='user_id', how='left')
      .join(tab2, on='user_id', how='left')
      .join(tab3, on='user_id', how='left')
      .join(tab4, on='user_id', how='left')
      .join(tab5, on='user_id', how='left')
      .join(tab6, on='user_id', how='left')
      .join(tab7, on='user_id', how='left')
      .join(tab8, on='user_id', how='left')
      .join(train, on='user_id', how='left')
     )

In [42]:
print(list(df.columns))

['report', 'employee_count_nm', 'bankemplstatus', 'customer_age', 'report_dt', 'ELSE', 'RUR', 'USD', 'EUR', 'tran_n', 'amt_sum', 'amt_max', 'amt_min', 'amt_mean', 'amt_q_10', 'amt_q_90', 'amt_median', 'amt_std', 'amt_kurtosis', 'amt_skew', 'mcc_nunique', 'last_day_diff', 'first_day_diff', 'life', 'interval_max', 'interval_min', 'interval_mean', 'interval_median', 'interval_q_10', 'interval_q_90', 'interval_kurtosis', 'interval_skew', 'interval_std', 'sp', 'sn', 'p_count', 'n_count', 'p_mean', 'n_mean', 'balance', 'mcc_logsumabs_[0]', 'mcc_logsumabs_[1]', 'mcc_logsumabs_[2]', 'mcc_logsumabs_[3]', 'mcc_logsumabs_[4]', 'mcc_logsumabs_[6]', 'mcc_logsumabs_[7]', 'mcc_logsumabs_[8]', 'mcc_logsumabs_[9]', 'mcc_logsumabs_[10]', 'mcc_logsumabs_[11]', 'mcc_logsumabs_[12]', 'mcc_logsumabs_[13]', 'mcc_logsumabs_[14]', 'mcc_logsumabs_[15]', 'mcc_logsumabs_[16]', 'mcc_logsumabs_[17]', 'mcc_logsumabs_[18]', 'mcc_logsumabs_[19]', 'mcc_logsumabs_[20]', 'mcc_logsumabs_[21]', 'mcc_logsumabs_[22]', 'mcc_l

In [43]:
df = df.drop(['report_dt'],axis=1)
df['employee_count_nm'] = df.employee_count_nm.fillna(-1).astype(int)
df = df.reset_index()

In [44]:
df.head(5)

,user_id,report,employee_count_nm,bankemplstatus,customer_age,ELSE,RUR,USD,EUR,tran_n,amt_sum,amt_max,amt_min,amt_mean,amt_q_10,amt_q_90,amt_median,amt_std,amt_kurtosis,amt_skew,mcc_nunique,last_day_diff,first_day_diff,life,interval_max,interval_min,interval_mean,interval_median,interval_q_10,interval_q_90,interval_kurtosis,interval_skew,interval_std,sp,sn,p_count,n_count,p_mean,n_mean,balance,mcc_logsumabs_[0],mcc_logsumabs_[1],mcc_logsumabs_[2],mcc_logsumabs_[3],mcc_logsumabs_[4],mcc_logsumabs_[6],mcc_logsumabs_[7],mcc_logsumabs_[8],mcc_logsumabs_[9],mcc_logsumabs_[10],...,w_count_[21],w_count_[22],w_count_[23],w_count_[24],w_count_[25],w_count_[26],w_count_[27],w_count_[28],w_count_[29],w_count_[30],w_count_[31],w_count_[32],w_count_[33],w_count_[34],w_count_[35],w_count_[36],w_count_[37],w_count_[38],w_count_[39],w_count_[40],m_logsumabs_[-9],m_logsumabs_[-8],m_logsumabs_[-7],m_logsumabs_[-6],m_logsumabs_[-5],m_logsumabs_[-4],m_logsumabs_[-3],m_logsumabs_[3],m_logsumabs_[4],m_logsumabs_[5],m_logsumabs_[6],m_logsumabs_[7],m_logsumabs_[8],m_logsumabs_[9],m_count_[-9],m_count_[-8],m_count_[-7],m_count_[-6],m_count_[-5],m_count_[-4],m_count_[-3],m_count_[3],m_count_[4],m_count_[5],m_count_[6],m_count_[7],m_count_[8],m_count_[9],target,time
0,3,2,4,0,3,0.00000,12.789712,0.0,0.0,11,13706.416641,104011.960938,-153866.890625,1246.037876,-15144.601562,37991.929688,4549.455078,60860.880477,5.091569,-1.399640,4,108,214,106,30.0,0.0,10.600000,7.5,0.0,21.9,-0.946217,0.603867,10.689559,64.222449,34.862271,7,4,5.838404,3.169297,29.360178,12.134084,0.000000,0.000000,8.128680,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0,0,0,2,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,11.833084,10.670715,9.925372,11.943843,8.128680,0.000000,0.000000,0,0,0,0,0,0,0,3,3,2,1,2,0,0,0.0,77.0
1,9,1,6,0,3,0.00000,12.686752,0.0,0.0,90,-323434.666813,-45.579891,-90147.617188,-3593.718520,-4270.121582,-219.823502,-840.509674,11797.055119,37.221420,-5.877426,22,102,283,181,22.0,0.0,2.033708,1.0,0.0,6.0,12.504731,3.340798,3.791286,0.000000,622.581098,0,90,0.000000,6.917568,-622.581098,0.000000,9.930944,9.026902,12.168032,7.417504,7.619438,0.0000,8.557285,0.000000,0.000000,...,11,2,0,0,4,3,11,2,5,1,3,6,1,3,2,1,1,3,11,3,0.0,0.0,0.0,0.0,9.825543,9.424449,9.313587,9.494728,11.554290,11.606416,10.899446,0.000000,0.000000,0.000000,0,0,0,0,12,9,14,8,15,9,23,0,0,0,NaN,NaN
2,13,6,5,0,2,9.28478,12.825245,0.0,0.0,22,-124717.379150,70322.828125,-58740.300781,-5668.971780,-23855.722852,10937.443262,-10529.004883,23507.953991,5.245836,1.172701,4,114,282,168,21.0,0.0,8.000000,8.0,0.0,18.0,-1.223967,0.457177,7.280110,73.236937,135.005210,8,14,3.328952,6.136600,-61.768272,0.000000,0.000000,0.000000,12.437070,0.000000,0.000000,8.6991,0.000000,9.287803,0.000000,...,0,0,1,1,0,1,0,0,5,1,0,1,0,2,1,0,1,1,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10.210958,10.546587,10.227409,11.997725,10.690730,11.063459,9.863950,0,0,0,0,0,0,0,2,3,2,6,4,4,1,0.0,86.0
3,37,5,6,0,2,0.00000,12.775178,0.0,0.0,315,-331859.599463,5487.140625,-35782.984375,-1053.522538,-1940.371729,-74.076974,-236.420776,3913.114835,41.551736,-6.201203,28,104,283,179,7.0,0.0,0.570064,0.0,0.0,1.0,14.055624,3.109092,0.947434,17.176832,1772.637183,2,313,0.054530,5.627420,-1755.460350,9.281618,10.489704,10.188235,12.277408,4.843160,6.482128,0.0000,6.938342,10.185608,6.828845,...,22,16,7,18,21,15,10,10,10,23,19,8,11,4,10,9,12,5,3,7,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10.395736,10.630183,11.192290,10.978041,11.152946,10.820914,10.250063,0,0,0,0,0,0,0,15,77,67,54,55,37,10,0.0,89.0
4,41,1,4,0,2,0.00000,11.595303,0.0,0.0,16,-108586.614166,-290.766998,-16841.208984,-6786.663385,-12649.590820,-531.802338,-6328.293701,5251.737982,-1.001280,-0.291520,5,103,256,153,31.0,0.0,10.200000,2.0,0.0,29.2,-1.427902,0.815989,13.039500,0.000000,132.182240,0,16,0.000000,8.261390,-132.182240,0.000000,6.606730,0.000000,11.159380,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0,0,2,1,0,0,4,0,0,0,

In [45]:
df.to_csv('dataset.csv', index=False)

# Подбор моделей

In [46]:
X = df[~df.target.isnull()].drop(columns=['target', 'time'])
X_test = df[df.target.isnull()].drop(columns=['target', 'time'])
y = df[~df.target.isnull()]['target'].astype(int)

In [47]:
X

,user_id,report,employee_count_nm,bankemplstatus,customer_age,ELSE,RUR,USD,EUR,tran_n,amt_sum,amt_max,amt_min,amt_mean,amt_q_10,amt_q_90,amt_median,amt_std,amt_kurtosis,amt_skew,mcc_nunique,last_day_diff,first_day_diff,life,interval_max,interval_min,interval_mean,interval_median,interval_q_10,interval_q_90,interval_kurtosis,interval_skew,interval_std,sp,sn,p_count,n_count,p_mean,n_mean,balance,mcc_logsumabs_[0],mcc_logsumabs_[1],mcc_logsumabs_[2],mcc_logsumabs_[3],mcc_logsumabs_[4],mcc_logsumabs_[6],mcc_logsumabs_[7],mcc_logsumabs_[8],mcc_logsumabs_[9],mcc_logsumabs_[10],...,w_count_[19],w_count_[20],w_count_[21],w_count_[22],w_count_[23],w_count_[24],w_count_[25],w_count_[26],w_count_[27],w_count_[28],w_count_[29],w_count_[30],w_count_[31],w_count_[32],w_count_[33],w_count_[34],w_count_[35],w_count_[36],w_count_[37],w_count_[38],w_count_[39],w_count_[40],m_logsumabs_[-9],m_logsumabs_[-8],m_logsumabs_[-7],m_logsumabs_[-6],m_logsumabs_[-5],m_logsumabs_[-4],m_logsumabs_[-3],m_logsumabs_[3],m_logsumabs_[4],m_logsumabs_[5],m_logsumabs_[6],m_logsumabs_[7],m_logsumabs_[8],m_logsumabs_[9],m_count_[-9],m_count_[-8],m_count_[-7],m_count_[-6],m_count_[-5],m_count_[-4],m_count_[-3],m_count_[3],m_count_[4],m_count_[5],m_count_[6],m_count_[7],m_count_[8],m_count_[9]
0,3,2,4,0,3,0.00000,12.789712,0.0,0.0,11,13706.416641,104011.960938,-153866.890625,1246.037876,-15144.601562,37991.929688,4549.455078,60860.880477,5.091569,-1.399640,4,108,214,106,30.0,0.0,10.600000,7.5,0.0,21.9,-0.946217,0.603867,10.689559,64.222449,34.862271,7,4,5.838404,3.169297,29.360178,12.134084,0.000000,0.000000,8.128680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,2,0,0,0,2,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,11.833084,10.670715,9.925372,11.943843,8.128680,0.000000,0.000000,0,0,0,0,0,0,0,3,3,2,1,2,0,0
2,13,6,5,0,2,9.28478,12.825245,0.0,0.0,22,-124717.379150,70322.828125,-58740.300781,-5668.971780,-23855.722852,10937.443262,-10529.004883,23507.953991,5.245836,1.172701,4,114,282,168,21.0,0.0,8.000000,8.0,0.0,18.0,-1.223967,0.457177,7.280110,73.236937,135.005210,8,14,3.328952,6.136600,-61.768272,0.000000,0.000000,0.000000,12.437070,0.000000,0.000000,8.699100,0.000000,9.287803,0.000000,...,1,2,0,0,1,1,0,1,0,0,5,1,0,1,0,2,1,0,1,1,1,1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.210958,10.546587,10.227409,11.997725,10.690730,11.063459,9.863950,0,0,0,0,0,0,0,2,3,2,6,4,4,1
3,37,5,6,0,2,0.00000,12.775178,0.0,0.0,315,-331859.599463,5487.140625,-35782.984375,-1053.522538,-1940.371729,-74.076974,-236.420776,3913.114835,41.551736,-6.201203,28,104,283,179,7.0,0.0,0.570064,0.0,0.0,1.0,14.055624,3.109092,0.947434,17.176832,1772.637183,2,313,0.054530,5.627420,-1755.460350,9.281618,10.489704,10.188235,12.277408,4.843160,6.482128,0.000000,6.938342,10.185608,6.828845,...,4,23,22,16,7,18,21,15,10,10,10,23,19,8,11,4,10,9,12,5,3,7,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.395736,10.630183,11.192290,10.978041,11.152946,10.820914,10.250063,0,0,0,0,0,0,0,15,77,67,54,55,37,10
4,41,1,4,0,2,0.00000,11.595303,0.0,0.0,16,-108586.614166,-290.766998,-16841.208984,-6786.663385,-12649.590820,-531.802338,-6328.293701,5251.737982,-1.001280,-0.291520,5,103,256,153,31.0,0.0,10.200000,2.0,0.0,29.2,-1.427902,0.815989,13.039500,0.000000,132.182240,0,16,0.000000,8.261390,-132.182240,0.000000,6.606730,0.000000,11.159380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4,0,0,0,2,1,0,0,4,0,0,0,1,1,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.000000,9.807809,9.731584,0.000000,8.992861,9.781772,10.320704,9.767794,0.000000,0.000000,0.000000,0,0,0,0,2,1,0,2,4,3,4,0,0,0
5,42,12,0,0,3,0.00000,11.806563,0.0,0.0,59,11429.587215,45706.445312,-19526.582031,193.721817,-2197.000293,2385.822656,-321.756958,6735.085596,38.130759,5.059008,20,102,283,181,33.0,0.0,3.120690,1.0,0.0,8.0,14.534027,3.585225,6.029395,88.890396,295.796686,11,48,1.506617,5.013503,-206.906290,6.793708,9.422266,7.790637,0.000000,8.231095,0.000000,6.947312,6.944826,10.264454,7.729518,...,0,2

In [48]:
X_test

,user_id,report,employee_count_nm,bankemplstatus,customer_age,ELSE,RUR,USD,EUR,tran_n,amt_sum,amt_max,amt_min,amt_mean,amt_q_10,amt_q_90,amt_median,amt_std,amt_kurtosis,amt_skew,mcc_nunique,last_day_diff,first_day_diff,life,interval_max,interval_min,interval_mean,interval_median,interval_q_10,interval_q_90,interval_kurtosis,interval_skew,interval_std,sp,sn,p_count,n_count,p_mean,n_mean,balance,mcc_logsumabs_[0],mcc_logsumabs_[1],mcc_logsumabs_[2],mcc_logsumabs_[3],mcc_logsumabs_[4],mcc_logsumabs_[6],mcc_logsumabs_[7],mcc_logsumabs_[8],mcc_logsumabs_[9],mcc_logsumabs_[10],...,w_count_[19],w_count_[20],w_count_[21],w_count_[22],w_count_[23],w_count_[24],w_count_[25],w_count_[26],w_count_[27],w_count_[28],w_count_[29],w_count_[30],w_count_[31],w_count_[32],w_count_[33],w_count_[34],w_count_[35],w_count_[36],w_count_[37],w_count_[38],w_count_[39],w_count_[40],m_logsumabs_[-9],m_logsumabs_[-8],m_logsumabs_[-7],m_logsumabs_[-6],m_logsumabs_[-5],m_logsumabs_[-4],m_logsumabs_[-3],m_logsumabs_[3],m_logsumabs_[4],m_logsumabs_[5],m_logsumabs_[6],m_logsumabs_[7],m_logsumabs_[8],m_logsumabs_[9],m_count_[-9],m_count_[-8],m_count_[-7],m_count_[-6],m_count_[-5],m_count_[-4],m_count_[-3],m_count_[3],m_count_[4],m_count_[5],m_count_[6],m_count_[7],m_count_[8],m_count_[9]
1,9,1,6,0,3,0.000000,12.686752,0.0,0.0,90,-323434.666813,-45.579891,-90147.617188,-3593.718520,-4270.121582,-219.823502,-840.509674,11797.055119,37.221420,-5.877426,22,102,283,181,22.0,0.0,2.033708,1.0,0.0,6.0,12.504731,3.340798,3.791286,0.000000,622.581098,0,90,0.000000,6.917568,-622.581098,0.000000,9.930944,9.026902,12.168032,7.417504,7.619438,0.000000,8.557285,0.000000,0.000000,...,0,6,11,2,0,0,4,3,11,2,5,1,3,6,1,3,2,1,1,3,11,3,0.0,0.000000,0.000000,0.000000,9.825543,9.424449,9.313587,9.494728,11.554290,11.606416,10.899446,0.000000,0.000000,0.000000,0,0,0,0,12,9,14,8,15,9,23,0,0,0
9,61,1,2,0,3,11.057102,11.915118,0.0,0.0,67,32793.204498,23699.917969,-11909.896484,489.450813,-4106.613232,8299.375293,-403.328491,5877.762624,4.784045,1.550408,13,105,260,155,18.0,0.0,2.333333,1.0,0.0,7.0,6.933100,2.582521,3.919707,114.103996,353.661739,13,54,1.703045,5.278533,-239.557743,11.376215,10.615527,7.592499,9.567973,7.898502,8.587591,0.000000,7.196587,9.351298,0.000000,...,7,3,1,6,3,1,0,1,8,10,0,0,1,2,6,2,3,2,2,0,0,0,0.0,0.000000,0.000000,0.000000,10.504544,11.066442,0.000000,10.096596,9.812663,9.607888,10.915607,0.000000,0.000000,0.000000,0,0,0,0,9,9,0,7,12,11,19,0,0,0
10,62,10,0,0,1,0.000000,10.928738,0.0,0.0,12,49056.019875,18678.558594,-1579.360107,4088.001656,-606.512354,17663.653711,-92.957352,8103.265670,-0.218417,1.338234,7,104,253,149,44.0,0.0,13.545455,7.0,0.0,31.0,-0.411767,0.902486,15.449036,29.297618,47.888081,3,9,2.441468,3.990673,-18.590463,10.866775,0.000000,5.934738,0.000000,5.351411,0.000000,0.000000,0.000000,0.000000,5.907158,...,0,0,0,0,2,4,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0.0,5.934738,7.757936,9.671791,9.835132,9.790523,0.000000,6.448746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,7,1,1,1,0,1,0,0,0,0,0,0
17,80,5,6,0,1,0.000000,13.589936,0.0,0.0,427,-572952.872180,50527.179688,-31925.750000,-1341.810005,-4826.099609,-121.263309,-758.616333,4144.089409,69.731426,3.545599,45,101,280,179,5.0,0.0,0.420188,0.0,0.0,1.0,7.500724,2.236888,0.699220,72.588888,2761.327255,8,419,0.169997,6.466809,-2688.738367,11.631175,12.618422,8.061558,10.686826,10.820789,5.675217,10.323193,6.396576,0.000000,9.303568,...,14,16,14,15,10,5,15,20,13,12,17,21,16,12,17,13,12,12,13,14,2,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.294358,11.572749,11.192578,11.478664,11.947540,11.659183,10.434862,0,0,0,0,0,0,0,111,73,48,68,72,52,3
20,88,3,-1,0,2,0.000000,10.819167,0.0,0.0,50,-49969.439034,-6.142165,-9443.094727,-999.388781,-1481.414197,-27.176533,-301.121384,2188.048153,10.471049,-3.353958,12,134,282,148,55.0,0.0,3.020408,0.0,0.0,6.2,33.107170,5.438531,8.320381,0.000000,278.239573,0,50,0.000000,5.564791,-278.239573,0.000000,9.549614,8.386387,6.981332,5.088642,0.000000,

In [49]:
y.value_counts()

target
0    58583
1     5417
Name: count, dtype: int64

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

### CatBoost

In [51]:
cat_cols = ['user_id', 'report', 'bankemplstatus', 'customer_age', 'employee_count_nm']

model1 = CatBoostClassifier(
    iterations = 1000,
    learning_rate = 0.03,
    depth = 9,
    custom_metric = ['AUC', 'F1'], 
    eval_metric = 'Logloss', 
    early_stopping_rounds = 50,
    random_state = 0
)

model1.fit(X_train, y_train, eval_set=(X_val, y_val), cat_features=cat_cols, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [52]:
model1.best_score_

{'learn': {'F1': 0.27624735118474286, 'Logloss': 0.20695707355506152},
 'validation': {'F1': 0.19206349206349208,
  'Logloss': 0.24133634954039995,
  'AUC': 0.7798721500707027}}

In [53]:
model1.get_feature_importance(prettified=True).head(30)

,Feature Id,Importances
0,employee_count_nm,5.707051
1,customer_age,4.043985
2,w_logsumabs_[15],2.175584
3,amt_std,1.888195
4,interval_median,1.768431
5,interval_skew,1.751961
6,last_day_diff,1.721584
7,w_logsumabs_[14],1.623576
8,life,1.621631
9,mcc_nunique,1.587614


In [54]:
importance = model1.get_feature_importance(prettified=True)
importance = importance[importance.Importances > 0]
importance.tail(30)

,Feature Id,Importances
510,mcc_count_[135],4.068097e-04
511,mcc_logsumabs_[234],4.024817e-04
512,mcc_count_[271],3.843911e-04
513,mcc_logsumabs_[206],3.744701e-04
514,mcc_logsumabs_[104],3.728785e-04
515,mcc_count_[92],3.615471e-04
516,mcc_logsumabs_[92],3.599251e-04
517,mcc_count_[245],3.575434e-04
518,mcc_logsumabs_[151],3.555493e-04
519,mcc_count_[161],2.984518e-04


In [55]:
y_pred = model1.predict(X_val)

In [56]:
print(confusion_matrix(y_val, y_pred, labels = [0, 1]))

[[11659    70]
 [  950   121]]


In [57]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96     11729
           1       0.63      0.11      0.19      1071

    accuracy                           0.92     12800
   macro avg       0.78      0.55      0.57     12800
weighted avg       0.90      0.92      0.89     12800



### SMOTE + CatBoost

In [58]:
os = SMOTE(random_state=0)
X_train1, y_train1 = os.fit_resample(X_train, y_train)

In [59]:
cat_cols = ['user_id', 'report', 'bankemplstatus', 'customer_age', 'employee_count_nm']

model2 = CatBoostClassifier(
    iterations = 1000,
    learning_rate = 0.03,
    depth = 9,
    custom_metric = ['AUC', 'F1'], 
    eval_metric = 'Logloss', 
    early_stopping_rounds = 50,
    random_state = 0
)

model2.fit(X_train1, y_train1, eval_set=(X_val, y_val), cat_features=cat_cols, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [60]:
model2.best_score_

{'learn': {'F1': 0.9603911115043855, 'Logloss': 0.1098196681992023},
 'validation': {'F1': 0.2513139034878165,
  'Logloss': 0.2458431533450486,
  'AUC': 0.7660414437181926}}

In [61]:
model2.get_feature_importance(prettified=True).head(30)

,Feature Id,Importances
0,first_day_diff,4.326193
1,w_logsumabs_[40],3.857634
2,m_logsumabs_[-6],2.553645
3,m_logsumabs_[6],2.281798
4,m_logsumabs_[8],2.239794
5,interval_median,2.207552
6,m_logsumabs_[7],1.812134
7,m_logsumabs_[-7],1.776884
8,m_logsumabs_[-4],1.703150
9,m_logsumabs_[-5],1.688413


In [62]:
importance = model2.get_feature_importance(prettified=True)
importance = importance[importance.Importances > 0]
importance.tail(30)

,Feature Id,Importances
507,mcc_logsumabs_[313],3.850038e-05
508,mcc_logsumabs_[119],3.810391e-05
509,mcc_count_[328],3.805298e-05
510,mcc_count_[61],3.667403e-05
511,mcc_logsumabs_[223],3.372313e-05
512,mcc_logsumabs_[220],3.322306e-05
513,mcc_count_[194],3.144955e-05
514,mcc_count_[233],3.102386e-05
515,mcc_count_[253],2.925479e-05
516,mcc_logsumabs_[251],2.640694e-05


In [63]:
y_pred = model2.predict(X_val)

In [64]:
print(confusion_matrix(y_val, y_pred, labels = [0, 1]))

[[11671    58]
 [  945   126]]


In [65]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     11729
           1       0.68      0.12      0.20      1071

    accuracy                           0.92     12800
   macro avg       0.80      0.56      0.58     12800
weighted avg       0.90      0.92      0.90     12800



### RandomOverSampler + CatBoost

In [66]:
X_train2 = X_train.drop('user_id', axis=1)

In [67]:
os = RandomOverSampler(random_state=0)
X_train2, y_train2 = os.fit_resample(X_train2, y_train)

In [68]:
cat_cols = ['report', 'bankemplstatus', 'customer_age', 'employee_count_nm']

model3 = CatBoostClassifier(
    iterations = 1000,
    learning_rate = 0.03,
    depth = 9,
    custom_metric = ['AUC', 'F1'], 
    eval_metric = 'Logloss', 
    early_stopping_rounds = 50,
    random_state = 0
)

model3.fit(X_train2, y_train2, eval_set=(X_val.drop('user_id', axis=1), y_val), cat_features=cat_cols, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [69]:
model3.best_score_

{'learn': {'F1': 0.9813309813309812, 'Logloss': 0.1967858614241168},
 'validation': {'F1': 0.35031847133757965,
  'Logloss': 0.32872545793794744,
  'AUC': 0.7717000461479957}}

In [70]:
model3.get_feature_importance(prettified=True).head(30)

,Feature Id,Importances
0,employee_count_nm,3.346235
1,customer_age,1.946568
2,amt_std,1.768630
3,mcc_nunique,1.546374
4,w_logsumabs_[15],1.515048
5,RUR,1.438347
6,interval_kurtosis,1.362871
7,life,1.331803
8,w_logsumabs_[18],1.278804
9,report,1.240689


In [71]:
importance = model3.get_feature_importance(prettified=True)
importance = importance[importance.Importances > 0]
importance.tail(30)

,Feature Id,Importances
397,mcc_logsumabs_[92],0.000821
398,mcc_logsumabs_[187],0.000815
399,mcc_count_[118],0.000800
400,mcc_count_[29],0.000791
401,mcc_logsumabs_[89],0.000769
402,mcc_logsumabs_[275],0.000755
403,mcc_logsumabs_[211],0.000745
404,mcc_logsumabs_[252],0.000735
405,mcc_logsumabs_[316],0.000661
406,mcc_logsumabs_[208],0.000649


In [72]:
y_pred = model3.predict(X_val.drop('user_id', axis=1))

In [73]:
print(confusion_matrix(y_val, y_pred, labels = [0, 1]))

[[10934   795]
 [  682   389]]


In [74]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94     11729
           1       0.33      0.36      0.35      1071

    accuracy                           0.88     12800
   macro avg       0.63      0.65      0.64     12800
weighted avg       0.89      0.88      0.89     12800



# Построение итоговой модели и прогноз

In [75]:
cat_cols = ['user_id', 'report', 'bankemplstatus', 'customer_age', 'employee_count_nm']

model = CatBoostClassifier(
    iterations = 1000,
    learning_rate = 0.03,
    depth = 9,
    custom_metric = ['AUC', 'F1'], 
    eval_metric = 'Logloss', 
    early_stopping_rounds = 50,
    random_state = 0
)

model.fit(X, y, eval_set=(X, y), cat_features=cat_cols, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [76]:
test_preds = pd.DataFrame(model.predict_proba(X_test), 
                          index = X_test['user_id'].values, 
                          columns=['pred_0','pred_1'])

In [77]:
test_preds

,pred_0,pred_1
9,0.974515,0.025485
61,0.967647,0.032353
62,0.956693,0.043307
80,0.995829,0.004171
88,0.807396,0.192604
...,...,...
561362,0.914123,0.085877
561419,0.938991,0.061009
561895,0.944281,0.055719
561908,0.917875,0.082125


In [78]:
test_preds['pred_1'].to_csv('submit6.csv', index_label=['user_id'], header=['predict'])